In [30]:
%%writefile get_bill_text.py
'''
Once data has been populated into Mongo database, this script will populate the bill text
in the 'body' field if text doesn't already exist. 
'''
from pymongo import MongoClient
from bs4 import BeautifulSoup
import requests
import threading
from random import randint
from time import sleep

from my_tools import write_json_file


def url_builder(record_url):
    '''
    Builds endpoint url from leg_url in mongo. Endpoint url should be the site that 
    contains the text version of the bill.
    
    Parameters: a mongo record
    
    Returns:    url
    '''
    url_root = record_url.rsplit('?')[0]
    return '{}/text?format=txt&r=1'.format(url_root)


def get_bill_text(site_url):
    '''
    Scrapes the page at url to return the text of the bill.
    
    Parameters: url
    
    Returns:    bill text, if it exists
    '''
    # included sleep time to mimick human user 
    sleep_time = randint(2, 5)
    sleep(sleep_time)

    req = requests.get(site_url)
    stat_code = req.status_code

    # if error in getting url, print and log the error
    if stat_code != 200:
        print('_______________')
        print('_______________')
        print('')
        print('\t{}'.format(site_url))
        print('\t\tError in retrieving bill text.')
        print('\t\tRequest Status Code: {}'.format(stat_code))
        errored_line = {'url': site_url, 'error': stat_code}
        write_json_file(errored_line, '../data/logs/bill_text_errors.jsonl')
        print('Error logged in ../data/logs/bill_text_errors.jsonl')

    if stat_code == 200:
        soup = BeautifulSoup(req.content, 'lxml')
        # print(soup.prettify())

        # if there is no text, print and log the error
        if soup.find('pre') is None:
            print('_______________')
            print('_______________')
            print('\t{}'.format(site_url))
            print('\t\tNo text available for scraping. Logging...')
            errored_line = {'url': site_url, 'error': 'no text available', 'process': 'bill text'}
            write_json_file(errored_line, '../data/logs/bill_text_errors.jsonl')
            
            return None


        # else scrape the text
        else:
            bill_txt = soup.find('pre').text
            bill_txt = ' '.join(bill_txt.split())

            return bill_txt


def update_mongo_body(txt, bill_issue, cong_id, collection):  
    '''
    Updates the body field in the mongo record specified by bill_issue (leg_id) and
    cong_id (congress_id) from db.collection with txt.
    
    Parameters: txt - the text of the bill
                bill_issue - value to filter on for key leg_id
                cong_id - value to filter on for key congress_id
                collection - the name of the mongo collection
                
    Returns:    None
    '''
    collection.update_one({'leg_id': bill_issue, 'congress_id': cong_id}, {'$set': {'body': txt}})


def initiate_process(year, collection):
    '''
    Initiates process from threads.
    '''
#     client = MongoClient() # defaults to localhost
#     db = client.bills
#     bill_info = db.bill_info

    print('--------------------')
    print('Cleaning up year {}'.format(year))
    year_str = str(year)
    records_to_populate = collection.find({'leg_url': {'$regex': 'http'}, 'intro_date': {'$regex': year_str}, 'body': None})
    record_count = collection.count_documents({'leg_url': {'$regex': 'http'}, 'intro_date': {'$regex': year_str}, 'body': None})
    print('--> Number of records with no text for year {}: {}'.format(year, record_count))
    
    if record_count > 0:
        for rec in records_to_populate:
            # get complete url using url_builder
            url = url_builder(rec['leg_url'])
            # get bill text
            bill_text = get_bill_text(url)

            # update mongo record with bill text
            bill_issue = rec['leg_id']
            cong_id = rec['congress_id']
            update_mongo_body(bill_text, bill_issue, cong_id, collection)

            
            r = collection.count_documents({'leg_url': {'$regex': 'http'}, 'intro_date': {'$regex': year_str}, 'body': None})
            if r%100 == 0:
                print('+++++++++')
                print('Year {}: {} records remaining with no text'.format(year, r))
                print('+++++++++')
    
                
if __name__ == '__main__':
    print('This script is populating bill text into Mongo threading four years at a time for those records without any text.')
    client = MongoClient() # defaults to localhost
    db = client.bills
    bill_info = db.bill_info

    # iterate through date range in reverse
    year_range = range(2007, 2019)[::-1]

    for y in year_range[::4]:
        t1 = threading.Thread(target=initiate_process, args=[y, bill_info])
        t2 = threading.Thread(target=initiate_process, args=[y-1, bill_info])
        t3 = threading.Thread(target=initiate_process, args=[y-2, bill_info])
        t4 = threading.Thread(target=initiate_process, args=[y-3, bill_info])
        
        t1.start()
        t2.start()
        t3.start()
        t4.start()

        t1.join()
        t2.join()
        t3.join()
        t4.join()
        
    print('-----------')
    print('-----------')
    print('Bill text populating complete!... DATA SCIENCE!!!')
    

Overwriting get_bill_text.py


In [4]:
# exploration
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.bills
bill_info = db.bill_info

print('Number of records in database: {}'.format(bill_info.find().count()))



Number of records in database: 60616


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  import sys


In [5]:
records_with_text = bill_info.find({'body': {'$regex': 'e'}})


In [24]:
records_with_text_count = bill_info.count_documents({'body': {'$regex': 'e'}})
records_with_text_count

10675

In [25]:
records_with_text[10670]

{'_id': ObjectId('5c26cd6d1417de25ef8b48a8'),
 'leg_id': 'H R 6420',
 'leg_type': 'BILL',
 'leg_url': 'https://www.congress.gov/bill/109th-congress/house-bill/6420?s=1&r=110993',
 'intro_date': '12/08/2006',
 'congress_id': '109',
 'desc': 'Tax Exempt Hospitals Responsibility Act of 2006',
 'sponsor': 'Thomas, William M.',
 'sponsor_party': 'R',
 'sponsor_state': 'CA',
 'sponsor_district': '22',
 'num_of_cosponsors': '0',
 'cosponsors_url': None,
 'cosponsors': None,
 'num_of_amendments': None,
 'committee': 'House - Ways and Means',
 'bill_status': 'Introduced',
 'body': "[Congressional Bills 109th Congress] [From the U.S. Government Publishing Office] [H.R. 6420 Introduced in House (IH)] 109th CONGRESS 2d Session H. R. 6420 To amend the Internal Revenue Code of 1986 to impose an excise tax on certain medical care providers that fail to provide a minimum level of charity medical care, and for other purposes. _______________________________________________________________________ IN TH

In [3]:
bill_info.find().count()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


60616

In [4]:
bill_info.find_one()

{'_id': ObjectId('5c26c6c51417de25ef8a5a30'),
 'leg_id': 'H R 7149',
 'leg_type': 'BILL',
 'leg_url': 'https://www.congress.gov/bill/115th-congress/house-bill/7149?s=1&r=251',
 'intro_date': '11/16/2018',
 'congress_id': '115',
 'desc': 'Protecting Married Seniors from Impoverishment Act',
 'sponsor': 'Dingell, Debbie',
 'sponsor_party': 'D',
 'sponsor_state': 'MI',
 'sponsor_district': '12',
 'num_of_cosponsors': '1',
 'cosponsors_url': 'https://www.congress.gov/bill/115th-congress/house-bill/7149/cosponsors?s=1&r=251&overview=closed#tabs',
 'cosponsors': None,
 'num_of_amendments': None,
 'committee': 'House - Energy and Commerce',
 'bill_status': 'Introduced',
 'body': None}